# Amazon Fraud Detector - Data Profiler Notebook 


### Dataset Guidance
-------

AWS Fraud Detector's Online Fraud Insights(OFI) model supports a flexible schema, enabling you to train an OFI model to your specific data and business need. This notebook was developed to help you profile your data and identify potenital issues before you train an OFI model. The following summarizes the minimimum CSV File requirements:

* The files are in CSV UTF-8 (comma delimited) format (*.csv).
* The file should contain at least 10k rows and the following __four__ required fields:  

    * Event timestamp 
    * IP address 
    * Email address
    * Fraud label 
    
* The maximum file size is 10 gigabytes (GB).  

* The following dates and datetime formats are supported:
    * Dates: YYYY-MM-DD (eg. 2019-03-21)
    * Datetime: YYYY-MM-DD HH:mm:ss (eg. 2019-03-21 12:01:32) 
    * ISO 8601 Datetime: YYYY-MM-DDTHH:mm:ss+/-HH:mm (eg. 2019-03-21T20:58:41+07:00)

* The decimal precision is up to four decimal places.
* Numeric data should not contain commas and currency symbols. 
* Columns with values that could contain commas, such as address or custom text should be enclosed in double quotes. 



### Getting Started with Data 
-------
The following general guidance is provided to get the most out of your AWS Fraud Detector Online Fraud Insights Model. 

* Gathering Data -  The OFI model requires a minimum of 10k records. We recommend that a minimum of 6 weeks of historic data is collected, though 3 - 6 months of data is preferable.   As part of the process the OFI model partitions your data based on the Event Timestamp such that performance metrics are calculated on the out of sample (latest) data, thus the format of the event timestamp is important. 

  
* Data & Label Maturity: As part of the data gathering process we want to insure that records have had sufficient time to “mature”, i.e. that enough time has passed to insure “non-fraud" and  “fraud” records have been correctly identified. It often takes 30 - 45 days (or more) to correctly identify fraudulent events, because of this it is important to insure that the latest records are at least 30 days old or older.  

  
* Sampling: The OFI training process will sample and partition historic based on event timestamp. There is no need to manually sample the data and doing so may negatively influence your model’s results.  

  
* Fraud Labels:  The OFI model requires that a minimum of 500 observations are identified and labeled as “fraud”. As noted above, fraud label maturity is important. Insure that extracted data has sufficiently matured to insure that fraudulent events have been reliably found. 
  
  
* Custom Fields: the OFI model requires 4 fields: event timestamp, IP address, email address and fraud label. The more custom fields you provide the better the OFI model can differentiate between fraud and not fraud.  
  
  
* Nulls and Missing Values: OFI model handles null and missing values, however the percentage of nulls in key fields should be limited. Especially timestamp and fraud label columns should not contain any missing values.   

  
If you would like to know more, please check out the [Fraud Detector's Documentation](https://docs.aws.amazon.com/frauddetector/). 


In [1]:
from IPython.core.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:90% }</style>"))
from IPython.display import IFrame
# ------------------------------------------------------------------
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.4f}'.format

# -- AWS stuff -- 
import boto3


### Amazon Fraud Detector Profiling 
-----

from github download and copy the afd_profile.py python program and template directory to your notebook  

<div class="alert alert-info"> <strong> afd_profile.py </strong>

- afd_profile.py - is the python package which will generate your profile report. 
- /templates - directory contains the supporting profile templates 


</div>


In [2]:
# -- get this package from github -- 
import afd_profile

### Intialize your S3 client 
-----
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html

In [3]:
client = boto3.client('s3')

### File & Field Mapping
-----
Simply map your file and field names to the required config values. 

<div class="alert alert-info"> <strong> Map the Required fields </strong>

- input_file: this is your CSV file in your s3 bucket 

<b> required_features </b> are the minimally required freatures to run Amazon Fraud Detector 
- EVENT_TIMESTAMP: map this to your file's Date or Datetime field.    
- IP_ADDRESS: map this to your file's IP address field.   
- EMAIL_ADDRESS: map this to your file's email address field.  
- FRAUD_LABEL: map this to your file's fraud label field.  
    **note: the profiler will identify the "rare" case and assume that it is fraud**
    
</div>


In [4]:
# input_file = 's3://sagemaker-us-east-2-057716757052/fraud/console/registration_data_20K_minimum.csv'
# ! aws s3 cp {input_file} .

In [7]:
local_input_file = '../data/registration_data_20K_full.csv'
# %store -r train_data_file_name
# %store -r test_data_file_name

## 기존 노트북에서 생성한 데이터 사용
# train_data_file_name = '../' + train_data_file_name
# test_data_file_name = '../' +  test_data_file_name

In [8]:
import pandas as pd
df = pd.read_csv(local_input_file)
df.head()

,ip_address,email_address,billing_state,user_agent,billing_postal,phone_number,EVENT_TIMESTAMP,billing_address,EVENT_LABEL
0,112.136.132.151,fake_cgonzales@example.net,NC,Mozilla/5.0 (iPad; CPU iPad OS 10_3_3 like Mac...,34491,(555)333 - 9246,2020-02-22 01:38:31,12351 Amanda Knolls Fake St.,legit
1,192.169.234.143,fake_dustin64@example.net,CO,Mozilla/5.0 (Windows; U; Windows NT 6.1) Apple...,34555,(555)779 - 5604,2020-04-08 23:29:41,691 Deborah Estate Fake St.,legit
2,185.112.224.79,fake_samuel59@example.net,CO,Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_7...,33611,(555)948 - 9198,2020-06-28 10:26:20,28583 Joseph Tunnel Fake St.,legit
3,68.73.183.186,fake_tinalopez@example.net,TN,Mozilla/5.0 (Linux; Android 7.1.1) AppleWebKit...,33520,(555)477 - 7885,2020-04-28 10:23:36,019 Linda Via Fake St.,legit
4,117.65.246.206,fake_alyssa53@example.com,NM,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,33889,(555)873 - 2599,2020-06-07 16:42:35,1023 Cook Loaf Fake St.,legit


In [9]:
# -- update your configuration -- 
config = {  
    "input_file"        : local_input_file,
    "required_features" : {
        "EVENT_TIMESTAMP" : "EVENT_TIMESTAMP",
        "EVENT_LABEL"     : "EVENT_LABEL",
        "IP_ADDRESS"      : "ip_address",
        "EMAIL_ADDRESS"   : "email_address"
    }
}


#### Run Profiler
-----
The profiler will read your file and produce an HTML file as a result which will be displayed inline within this notebook.  
  
Note: you can also open **report.html** in a separate browser tab. 

In [10]:
# -- generate the report object --
report = afd_profile.profile_report(config)

0


In [11]:
with open("report_full_cols.html", "w") as file:
    file.write(report)

IFrame(src='report_full_cols.html', width=1500, height=1500)
